In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import time

from ipywidgets import interact
from ipywidgets.widgets import IntSlider, fixed

Source: http://home.agh.edu.pl/~kzajac/dydakt/mownit/lab10/pde.pdf

Membrana w kształcie kwadratu o boku a jest równomiernie obciążona.
Membrana znajduje sie w stanie statycznym, a jej brzeg jest sztywno zamocowany i nieodkształcony.
Obliczyc odkształcenie membrany od poziomu , zakładając ze spełnia ono
równanie Poissona:

$$
\frac{\partial^2M}{\partial x^2} + \frac{\partial^2M}{\partial y^2} = \frac{-p}{T}
$$

Gdzie mamy ustalone parametry:
* $p>0$ - ciśnienie wywierane na membrane.
* $T$ - napięcie membrany.

$m$ - wartość napięcia membrany

Oznaczenia:
* $N$ := wymiary siatki
* $d = \frac{a}{N-1}$ := odległość między polami na siatce
* M := macierz $(N) \times (N)$ wartości napięcia membrany w danych punktach

$$
\frac{\partial^2 M}{\partial x^2} = \frac{M_{i+1,j} - 2 \cdot M_{ij} + M_{i-1,j} }{d^2}
$$

$$
\frac{\partial^2 M}{\partial y^2} = \frac{M_{i,j+1} - 2 \cdot M_{ij} + M_{i,j-1} }{d^2}
$$

Zapiszmy układ równań różniczkowych:

$$
\begin{cases}
    \forall i, j \in {1..N} : M_{1,j} = M_{N,j} = M_{i,1} = M_{i,N} = 0 \space\space \text{(krawędzie)} \\
    M_{i+1,j}  + M_{i-1,j} + M_{i,j+1} + M_{i,j-1} - 4 \cdot M_{ij} = \dfrac{-p \cdot d^2}{T} \space\space \text{(pozostałe punkty)}
\end{cases}
$$

W celu zwektoryzowania siatki M, wprowadźmy wektor $\theta$:

$$
M_{ij} = \theta_{N\cdot i + j}
$$

Zauważmy, że dla danego $M{ij} = \theta_k$
$$
\begin{cases}
    M_{i-1, j} = \theta_{k - N} \\
    M_{i+1, j} = \theta_{k + N} \\
    M_{i, j-1} = \theta_{k-1} \\
    M_{i, j+1} = \theta_{k+1}
\end{cases}
$$

Po przekształceniu:

$$
\begin{cases}
    \forall i \in (1 \space .. \space N) : \theta_i = 0 
        \space\space\text{(krawędź górna)}\\
    \forall i \in (1 \space .. \space N) : \theta_{N(N-1) + i} = 0
        \space\space\text{(krawędź dolna)}\\
    \forall j \in (0 \space .. \space N-1) : \theta_{Nj + 1} = 0 
        \space\space\text{(krawędź lewa)}\\
    \forall j \in (1 \space .. \space N) : \theta_{Nj} = 0
        \space\space\text{(krawędź prawa)}\\
    \theta_{k-N}  + \theta_{k+N} + \theta_{k-1} + \theta_{k+1} - 4 \cdot \theta_{k} = \dfrac{-p \cdot d^2}{T}
        \space\space\text{(pozostałe punkty)}\\
\end{cases}
$$


In [ ]:
a = 1.0
p = 1.0
T = 1.0

N = 30
d = a / (N - 1)

a, p, T, N, d
NN = N ** 2 # number of datapoints

Indeksy punktów na brzegach:

In [ ]:
def edges(N):
    NN = N ** 2
    up_edge = np.arange(0,N)
    down_edge = np.arange(NN - N, NN)
    left_edge = np.arange(N) * N
    right_edge = (np.arange(N) + 1) * N - 1
    return np.concatenate([up_edge, down_edge, left_edge, right_edge])

edges(N)

Wektor prawej strony układu równań:

In [ ]:
def results(N, p, d, T):
    NN = N ** 2 
    result = np.array([(-p * d ** 2) / T for _ in range(NN)])
    result[edges(N)] = 0
    return result

Macierz współczynników układu równań:

In [ ]:
def matrix(N):
    edg = edges(N)
    NN = N ** 2
    coeff = np.eye(NN) * -4
    coeff[np.arange(1, NN), np.arange(0, NN - 1)] = 1 # elem to the left
    coeff[np.arange(0, NN - 1), np.arange(1, NN)] = 1 # elems to the right
    coeff[np.arange(N, NN), np.arange(0, NN - N)] = 1 # elems to the up
    coeff[np.arange(0, NN - N - 1), np.arange(N, NN - 1)] = 1 # elems to the down
    coeff[:, edg] = 0
    coeff[edg, edg] = 1
    return coeff

matrix(3)

Wektor rozwiązań układu równań (od razu przekształcony, aby rozwiązania umieścić na siatce punktów)
Układ równań jest tu rozwiązywany za pomocą metody $np.linalg.solve$ mającej pod spodem biliotekę LAPACK

In [ ]:
def membrane_lapack(N, p, d, T):
    m = matrix(N)
    r = results(N, p, d, T)
    thetas = np.linalg.solve(m, r)
    return thetas.reshape(N,N)

In [ ]:
m_lapack = membrane_lapack(N, p, d, T)
sns.heatmap(m_lapack)

Pomiar czasu rozwiązania w zależności od przyjętego wymiaru siatki:

In [ ]:
Ns = np.arange(3, 104, 5)
trials = 10
Ns

In [ ]:
def membrane_performace(membrane_f, Ns=Ns, trials=trials):
    df = pd.DataFrame(columns=['N', 'solution_time'])

    for N in Ns:
        for t in range(trials):
            t1 = time.time()
            membrane_lapack(N, p, d, T)
            t2 = time.time()
            t_d = t2 - t1
            df = df.append({
                'N': N,
                'solution_time': t_d
                           },
            ignore_index=True)    
            print('size: ' + str(N) + ', trial: ' + str(t) + ", time: " + str(t_d), end="\r")
    return df

In [ ]:
lapack_csv = 'lapack.csv'

In [ ]:
# lapack_df = membrane_performace(membrane_lapack)
# lapack_df.to_csv('lapack.csv')

In [ ]:
lapack_df = pd.read_csv(lapack_csv)
lapack_df_agg = lapack_df.groupby('N').agg(['mean', 'std'])['solution_time']
lapack_df_agg

Wykres czasu znalezienia rozwiązania od wielkości siatki. Należy pamiętać, że jeżeli wymiar siatki to $N$, to punktów do obliczenia było $N^2$

In [ ]:
plt.scatter(lapack_df_agg['mean'].keys() ** 2, lapack_df_agg['mean'])
plt.errorbar(lapack_df_agg['mean'].keys() ** 2, lapack_df_agg['mean'], yerr=lapack_df_agg['std'])

plt.show()

In [ ]:
def disp_complexity(degree, df_agg):
    X_b = df_agg['mean'].keys() ** 2
    Y_b = df_agg['mean']
    Y_e = df_agg['std']
    p = np.polyfit(X_b, Y_b, degree)
    X = np.array([X_b ** i for i in range(degree, -1, -1)]).T
    Y = X @ p
    print(p)
    print(((Y_b - Y) ** 2).sum())
    plt.scatter(X_b, Y_b)
    plt.errorbar(X_b, Y_b, Y_e)
    plt.plot(X_b, Y, color='red')
    plt.show()

In [ ]:
interact(
    disp_complexity,
    degree=IntSlider(min=0, max=10),
    df_agg=fixed(lapack_df_agg)
)

Czas rozwiązania wydaje się rosnąć ze złożonością $O(n^3)$. Nie jest to ładne. Wypróbuję zatem iteracyjną metodę Jacobiego.

In [ ]:
M = np.array([[1,2], [3,4]])
D = M.diagonal() * np.eye(M.shape[0])
M - D

Solver nie jest skomplikowany:

In [ ]:
def jacobi_solve(matrix, results, iterations=1000, conv=1e-7):
    X = np.zeros_like(results)
    D = matrix.diagonal() * np.eye(matrix.shape[0]) 
    R = matrix - D
    Dinv = np.linalg.inv(D)
    for it in range(iterations):
#         print('iteration', it, "/", iterations, end="\r")
        X_tmp = Dinv @ (results - R @ X)
#         if (X - X_tmp).sum() < conv:
#             break
        X = X_tmp
    return X

Funkcja analogiczna do $membane\_lapack$, która używa solvera Jacobiego

In [ ]:
def membrane_jacobi(N, p, d, T, iterations=1000):
    m = matrix(N)
    r = results(N, p, d, T)
    thetas = jacobi_solve(m, r, iterations)
    return thetas.reshape(N,N)

In [ ]:
m_jacobi = membrane_jacobi(N, p, d, T)
sns.heatmap(m_jacobi)

In [ ]:
def jacobi_iters(N, iters):
    t0 = time.time()
    lapack = membrane_lapack(N, p, d, T)
    t1 = time.time()
    jacobi = membrane_jacobi(N, p, d, T, iters)
    t2 = time.time()
    print('N:',N)
    print('iterations:', iters)
    print('total error:', ((lapack - jacobi) ** 2).sum())
    print('lapack', t1-t0)
    sns.heatmap(lapack)
    plt.show()
    print('jacobi', t2-t1)
    sns.heatmap(jacobi)
    plt.show()

Porównajmy oba solvery:

In [ ]:
interact(
    jacobi_iters,
    N=IntSlider(min=3, max=100, value=30),
    iters=IntSlider(min=1, max=500, value=100)
)

In [ ]:
jacobi_csv = 'jacobi.csv'

In [ ]:
# jacobi_df = membrane_performace(membrane_jacobi)
# jacobi_df.to_csv(jacobi_csv)

In [ ]:
jacobi_df = pd.read_csv(jacobi_csv)
jacobi_df_agg = jacobi_df.groupby('N').agg(['mean', 'std'])['solution_time']
jacobi_df_agg

In [ ]:
plt.scatter(jacobi_df_agg['mean'].keys() ** 2, jacobi_df_agg['mean'], color='red')
plt.errorbar(jacobi_df_agg['mean'].keys() ** 2, jacobi_df_agg['mean'], yerr=jacobi_df_agg['std'], color='red')

plt.scatter(lapack_df_agg['mean'].keys() ** 2, lapack_df_agg['mean'], color='blue')
plt.errorbar(lapack_df_agg['mean'].keys() ** 2, lapack_df_agg['mean'], yerr=lapack_df_agg['std'], color='blue')

plt.show()

In [ ]:
interact(
    disp_complexity,
    degree=IntSlider(min=0, max=10),
    df_agg=fixed(jacobi_df_agg)
)